# Entendendo e Preparando uma base de dados para análise

### Importando a base de clientes de um banco de dados com o resultadodo pagamento ou não de um empréstimo

In [1]:
import pandas as pd
import re

In [2]:
clientes = pd.read_excel('ChavesClientes.xlsx')

In [3]:
clientes.head()

# verificando a base de dados que foi fornecida

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento
0,1,32FC,Ccinza,Basic-Alpha,1
1,2,25MV,AAmarelo,Black,1
2,3,27MV,B-Amarelo,Basic-Beta,1
3,4,26FD,BAmarelo,Black,0
4,5,26FD,C-Amarelo,Black,0


### Verificando a cardinalidade dos dados 

In [4]:
clientes.groupby(['Pagamento', 'ChaveSituacao'])['Pagamento'].count()

Pagamento  ChaveSituacao
0          26FD             2
           28FC             2
           28MD             1
           30FC             1
           31MD             1
1          25FD             1
           25FV             1
           25MV             1
           26MC             2
           27MC             1
           27MD             2
           27MV             1
           28FS             1
           29MV             1
           31MV             1
           32FC             1
Name: Pagamento, dtype: int64

### Ao analisar essa base, foi percebido a alta cardinalidade e por isso foi preciso tratar esse dados

#### comforme o questionamento com a área de negócios, a mesma informou o que cada coluna representa
#### ChaveSituacao: formada por
   ###### Idade do cliente (a idade miníma para ser cliente é 18 anos)
   ###### M: masculino
   ###### F: feminino
#### Estado civil do cliente:
   ###### S: solteiro
   ###### C: casado
   ###### D: divorciado
   ###### V: viúvo
#### ClasseRisco: formado por
   ###### Classificação do cliente como (A, B, C) e indicador (+, -, vazio)
   ###### Cor do cliente de acordo com o modelo interno da empresa
#### CatCliente: formado por
   ###### Categoria do cartão: qual o tipo de cartão do cliente
   ######     Basic
   ######     Black
   ######     Platinum
   ###### Categoria VIP: categoria do cliente VIP (caso exista)
   ######     Alpha
   ######     Beta

### Começando o tratamento com a separação da string na coluna

In [5]:
clientes['Idade'] = clientes.ChaveSituacao.str[:2]
clientes['Genero'] = clientes.ChaveSituacao.str[2:3]
clientes['EstadoCivil'] = clientes.ChaveSituacao.str[-1]

In [6]:
clientes.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C
1,2,25MV,AAmarelo,Black,1,25,M,V
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V
3,4,26FD,BAmarelo,Black,0,26,F,D
4,5,26FD,C-Amarelo,Black,0,26,F,D


### Fazendo um split para tratar a coluna CatCliente

In [7]:
# fazendo a separação usando um delimitador

clientes['Categoria'] = clientes.CatCliente.str.split('-').str.get(0)
clientes['VIP'] = clientes.CatCliente.str.split('-').str.get(1)

In [8]:
clientes.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil,Categoria,VIP
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C,Basic,Alpha
1,2,25MV,AAmarelo,Black,1,25,M,V,Black,NaN
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V,Basic,Beta
3,4,26FD,BAmarelo,Black,0,26,F,D,Black,NaN
4,5,26FD,C-Amarelo,Black,0,26,F,D,Black,NaN


In [9]:
# separando a coluna ClassRisco 

clientes['Risco'] = clientes.ClassRisco.apply(lambda x: re.findall('^[A-Z][^A-Za-z]?',x)[0])

In [10]:
clientes.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil,Categoria,VIP,Risco
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C,Basic,Alpha,C
1,2,25MV,AAmarelo,Black,1,25,M,V,Black,NaN,A
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V,Basic,Beta,B-
3,4,26FD,BAmarelo,Black,0,26,F,D,Black,NaN,B
4,5,26FD,C-Amarelo,Black,0,26,F,D,Black,NaN,C-


In [11]:
# verificando a base 

clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             20 non-null     int64 
 1   ChaveSituacao  20 non-null     object
 2   ClassRisco     20 non-null     object
 3   CatCliente     20 non-null     object
 4   Pagamento      20 non-null     int64 
 5   Idade          20 non-null     object
 6   Genero         20 non-null     object
 7   EstadoCivil    20 non-null     object
 8   Categoria      20 non-null     object
 9   VIP            8 non-null      object
 10  Risco          20 non-null     object
dtypes: int64(2), object(9)
memory usage: 1.8+ KB


### Transformando a coluna Idade em numérico

In [12]:
clientes['Idade'] = pd.to_numeric(clientes['Idade'])

In [13]:
clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             20 non-null     int64 
 1   ChaveSituacao  20 non-null     object
 2   ClassRisco     20 non-null     object
 3   CatCliente     20 non-null     object
 4   Pagamento      20 non-null     int64 
 5   Idade          20 non-null     int64 
 6   Genero         20 non-null     object
 7   EstadoCivil    20 non-null     object
 8   Categoria      20 non-null     object
 9   VIP            8 non-null      object
 10  Risco          20 non-null     object
dtypes: int64(3), object(8)
memory usage: 1.8+ KB


### Tratando valores vazios

In [14]:
clientes.loc[clientes.VIP.isnull(), 'VIP'] = 'Comum'

In [15]:
clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             20 non-null     int64 
 1   ChaveSituacao  20 non-null     object
 2   ClassRisco     20 non-null     object
 3   CatCliente     20 non-null     object
 4   Pagamento      20 non-null     int64 
 5   Idade          20 non-null     int64 
 6   Genero         20 non-null     object
 7   EstadoCivil    20 non-null     object
 8   Categoria      20 non-null     object
 9   VIP            20 non-null     object
 10  Risco          20 non-null     object
dtypes: int64(3), object(8)
memory usage: 1.8+ KB


### Criando uma nova base somente com as colunas necessárias para a análise

In [16]:
clientes.groupby(['Pagamento', 'Categoria'])['Pagamento'].count()

Pagamento  Categoria
0          Basic        3
           Black        3
           Platinum     1
1          Basic        3
           Black        4
           Platinum     6
Name: Pagamento, dtype: int64